<a href="https://colab.research.google.com/github/AkashNavgire02/Messy-vs-Clenan-room-classification/blob/main/Messy_vs_clean_room.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
train_dir = "/content/drive/My Drive/New folder/Messy vs clean room/images/train"
validation_dir = "/content/drive/My Drive/New folder/Messy vs clean room/images/val"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150)) 




Found 192 images belonging to 2 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))

Found 20 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='selu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='selu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='selu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(3000, activation='selu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(2000, activation='selu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1500, activation='selu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1000, activation='selu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(500, activation='selu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(300, activation='selu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(200, activation='selu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='selu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(50, activation='selu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='selu'),
    tf.keras.layers.Dropout(0.2),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('Organic') 
    # and 1 for the other ('Recycled')
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, 
                        verbose=1, mode='auto',restore_best_weights=True)
model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['acc'])

In [ ]:
history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=50,epochs=10,
                              validation_steps=5,
                              verbose=2, callbacks=[monitor])

Epoch 1/10
10/10 - 10s - loss: 1.2023 - acc: 0.5104 - val_loss: 0.7066 - val_acc: 0.5000
Epoch 2/10
10/10 - 10s - loss: 0.9475 - acc: 0.6458
Epoch 3/10
10/10 - 10s - loss: 0.8580 - acc: 0.6927
Epoch 4/10
10/10 - 10s - loss: 0.8473 - acc: 0.6562
Epoch 5/10
10/10 - 10s - loss: 0.8431 - acc: 0.7292
Epoch 6/10
10/10 - 10s - loss: 0.9227 - acc: 0.7083
Epoch 7/10
10/10 - 10s - loss: 0.4502 - acc: 0.8281
Epoch 8/10
10/10 - 10s - loss: 0.6599 - acc: 0.7917
Epoch 9/10
10/10 - 10s - loss: 0.4191 - acc: 0.8542
Epoch 10/10
10/10 - 10s - loss: 0.3348 - acc: 0.8906


In [ ]:
test2_datagen  = ImageDataGenerator( rescale = 1.0/255. )
test_dir = "/content/drive/My Drive/New folder/Messy vs clean room/images/test"
test_generator =  test2_datagen.flow_from_directory(test_dir,
                                                    batch_size=3,
                                                    class_mode  = 'input',
                                                    target_size = (150, 150),
                                                    shuffle=False)

Found 10 images belonging to 1 classes.


In [ ]:
y_prob = model.predict(test_generator)
y_prob





array([[0.97520846],
       [0.11275363],
       [0.9610413 ],
       [0.98931754],
       [0.9998449 ],
       [0.9966086 ],
       [0.38185948],
       [0.99459076],
       [0.9995593 ],
       [0.40799528]], dtype=float32)

In [ ]:
y_pred = ["Messy" if probs > 0.5 else "Clean" for probs in y_prob]
y_pred

['Messy',
 'Clean',
 'Messy',
 'Messy',
 'Messy',
 'Messy',
 'Clean',
 'Messy',
 'Messy',
 'Clean']